In [1]:
import org.apache.spark.sql.functions._


val source_data = Seq(
    (1, "A"), 
    (2, "B"), 
    (3, "C"), 
    (4, "D")
).toDF("source_id", "source_name")

val target_data = Seq(
    (1, "A"), 
    (2, "B"), 
    (4, "X"), 
    (5, "F")
).toDF("target_id", "target_name")

source_data.show(false)
target_data.show(false)



Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.101:4040
SparkContext available as 'sc' (version = 3.4.1, master = local[*], app id = local-1713022415413)
SparkSession available as 'spark'


+---------+-----------+
|source_id|source_name|
+---------+-----------+
|1        |A          |
|2        |B          |
|3        |C          |
|4        |D          |
+---------+-----------+

+---------+-----------+
|target_id|target_name|
+---------+-----------+
|1        |A          |
|2        |B          |
|4        |X          |
|5        |F          |
+---------+-----------+



source_data: org.apache.spark.sql.DataFrame = [source_id: int, source_name: string]
target_data: org.apache.spark.sql.DataFrame = [target_id: int, target_name: string]


In [2]:
val df = source_data.join(target_data, $"source_id"===$"target_id", "full_outer")

df.show(false)

+---------+-----------+---------+-----------+
|source_id|source_name|target_id|target_name|
+---------+-----------+---------+-----------+
|1        |A          |1        |A          |
|2        |B          |2        |B          |
|3        |C          |null     |null       |
|4        |D          |4        |X          |
|null     |null       |5        |F          |
+---------+-----------+---------+-----------+



df: org.apache.spark.sql.DataFrame = [source_id: int, source_name: string ... 2 more fields]


In [8]:
df.withColumn("comment", when($"target_name".isNull, "New in source"
                             ).when($"source_name".isNull, "New in target"
                                   ).when($"source_name" =!= $"target_name", "Mis-match")
             ).where($"comment".isNotNull
                    ).withColumn("ID", when($"source_id".isNull, $"target_id").otherwise($"source_id")
                                ).select("ID","comment").show(false)

+---+-------------+
|ID |comment      |
+---+-------------+
|3  |New in source|
|4  |Mis-match    |
|5  |New in target|
+---+-------------+



In [9]:
df.withColumn("comment", when($"target_name".isNull, "New in source"
                             ).when($"source_name".isNull, "New in target"
                                   ).when($"source_name" =!= $"target_name", "Mis-match")
             ).where($"comment".isNotNull
                    ).withColumn("ID", coalesce($"source_id", $"target_id")).select("ID","comment").show(false)

+---+-------------+
|ID |comment      |
+---+-------------+
|3  |New in source|
|4  |Mis-match    |
|5  |New in target|
+---+-------------+

